In [1]:
!pip install scipy openai tiktoken retry dashscope loguru

Looking in indexes: https://mirrors.cernet.edu.cn/pypi/web/simple

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install --upgrade pip

Looking in indexes: https://mirrors.cernet.edu.cn/pypi/web/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [3]:
!pip install scipy openai tiktoken retry dashscope loguru

Looking in indexes: https://mirrors.cernet.edu.cn/pypi/web/simple


In [5]:
import json
import os
from pprint import pprint
import re
from tqdm import tqdm
import random

import uuid
import openai
import tiktoken
import json
import numpy as np
import requests
from retry import retry
from scipy import sparse
#from rank_bm25 import BM25Okapi
#import jieba
from http import HTTPStatus
import dashscope

from concurrent.futures import ThreadPoolExecutor, as_completed
from loguru import logger
import json
import time
from tqdm import tqdm

dashscope.api_key="sk-b303026c4f004b48b7005dea911202ae"

logger.remove()  # 移除默认的控制台输出
logger.add("logs/app_{time:YYYY-MM-DD}.log", level="INFO", rotation="00:00", retention="10 days", compression="zip")

MODEL_NAME = 'qwen2-7b-instruct'

In [6]:
def call_qwen_api(MODEL_NAME, query):
    # 这里采用dashscope的api调用模型推理，通过http传输的json封装返回结果
    messages = [
        {'role': 'user', 'content': query}]
    response = dashscope.Generation.call(
        MODEL_NAME,
        messages=messages,
        result_format='message',  # set the result is message format.
    )
    if response.status_code == HTTPStatus.OK:
        print(response)
        return response['output']['choices'][0]['message']['content']
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        raise Exception()

In [7]:
def api_retry(MODEL_NAME, query):
    # 最大尝试次数
    max_retries = 5
    # 再次尝试等待时间
    retry_delay = 60  # in seconds
    attempts = 0
    while attempts < max_retries:
        try:
            return call_qwen_api(MODEL_NAME, query)
        except Exception as e:
            attempts += 1   
            if attempts < max_retries:
                logger.warning(f"Attempt {attempts} failed for text: {query}. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                logger.error(f"All {max_retries} attempts failed for text: {query}. Error: {e}")
                raise

In [8]:
# 这里定义了prompt推理模版

def get_prompt(problem, question, options):

    options = '\n'.join(f"{'ABCDEFG'[i]}. {o}" for i, o in enumerate(options))

    prompt = f"""你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为"答案是：A"。题目如下：

### 题目:
{problem}

### 问题:
{question}
{options}
"""
    # print(prompt)
    return prompt


In [9]:
# 这里使用extract抽取模获得抽取的结果

def extract(input_text):
    ans_pattern = re.compile(r"答案是：(.)", re.S)

    problems = ans_pattern.findall(input_text)
    # print(problems)
    if(problems == ''):
        return 'A'
    return problems[0]

In [10]:
def process_datas(datas,MODEL_NAME):
    results = []
    # 定义线程池 选择16线程
    with ThreadPoolExecutor(max_workers=16) as executor:
        # 这里我们使用future_data 存储每个线程的数据
        future_data = {}
        # 这里的lens记录了调用api的次数，也就是我们每个问题背景下的所有子问题之和。
        lens = 0
        # 送入多线程任务
        # 这里每个data下是一个问题背景，其中包含多个子问题。
        for data in tqdm(datas, desc="Submitting tasks", total=len(datas)):
            problem = data['problem']
            # 这里面我们用enumerate方法每次循环得到问题的序号id和实际的问题。
            for id,question in enumerate(data['questions']):
                prompt = get_prompt(problem, 
                                    question['question'], 
                                    question['options'],
                                    )
                # 这里送入线程池等待处理，使用api_retry，向api_retry传入MODEL_NAME, prompt参数
                future = executor.submit(api_retry, MODEL_NAME, prompt)
                # 每个线程我们存储对应的json问题数据以及问题序号id，这样我们就能定位出执行的是哪个子问题
                future_data[future] = (data,id)
                time.sleep(0.6)  # 控制每0.6秒提交一个任务 防止接口超过并发数
                lens += 1
        # 处理多线程任务
        for future in tqdm(as_completed(future_data), total=lens, desc="Processing tasks"):
            # print('data',data)
            # 取出每个线程中的字典数据及对应的问题id
            data = future_data[future][0]
            problem_id = future_data[future][1]
            try:
                # 获取api运行结果
                res  = future.result()
                # 抽取大语言模型返回结果
                extract_response = extract(res)
                # print('res',extract_response)
                # 装入answer字段
                data['questions'][problem_id]['answer'] = extract_response
                # 在结果列表中新增数据字典
                results.append(data)
                # print('data',data)
                
            except Exception as e:
                logger.error(f"Failed to process text: {data}. Error: {e}")
    
    return results

In [12]:
def main(ifn, ofn):
    if os.path.exists(ofn):
        pass
    data = []
    # 按行读取数据
    with open(ifn) as reader:
        for line in reader:
            sample = json.loads(line)
            data.append(sample)
    datas = data
    # print(data)
    # 均匀地分成多个数据集
    return_list = process_datas(datas,MODEL_NAME)
    print(len(return_list))
    print("All tasks finished!")
    return return_list
    
if __name__ == '__main__':
# 这里给了一个抽取范例参考
    a = extract("""根据欧几里得算法，逐步解析计算两个数6和7的最大公约数（gcd）的步骤如下：

1. 判断6和7是否相等：不相等。
2. 判断6和7大小关系，7 > 6，所以用更大的数7减去较小的数6得到结果1。
3. 现在计算6和1的最大公约数。
4. 6 > 1，根据算法用更大的数6减去较小的数1得到结果5。
5. 再计算5和1的最大公约数。
6. 5 > 1，用5减去1得到结果4。
7. 再计算4和1的最大公约数。
8. 4 > 1，用4减去1得到结果3。
9. 再计算3和1的最大公约数。
10. 3 > 1，用3减去1得到结果2。
11. 再计算2和1的最大公约数。
12. 2 > 1，用2减去1得到结果1。
13. 最后计算1和1的最大公约数，两数相等，gcd即为这两个数，也就是1。

因此，6和7的最大公约数是1。

答案是：C.""")

    print(a)
    # 调用主函数
    return_list = main('round1_test_data.jsonl', 'upload.jsonl')


C


Submitting tasks:   0%|          | 1/500 [00:00<05:00,  1.66it/s]

{"status_code": 200, "request_id": "a1e01922-1eea-9fb8-9e16-9b1635cc32a0", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "分析已知信息：\n- 我们知道鸡肉和苹果是食物。\n- Bill存活并吃了花生。\n- John吃所有食物，因此他吃了鸡肉、苹果和花生。\n- Sue吃所有Bill吃的食物，所以她吃了鸡肉、苹果和花生。\n- John喜欢所有食物，并且已经吃了这些食物。\n\n根据以上信息，我们可以得出结论：\n- Bill吃了花生。\n- Sue也吃了花生，因为她吃所有Bill吃的食物。\n- John吃了花生，因为他吃所有食物。\n- 所以，至少Bill、Sue和John都吃了花生。\n\n答案是：D. None of the above"}}]}, "usage": {"input_tokens": 210, "output_tokens": 128, "total_tokens": 338}}


Submitting tasks:   1%|          | 3/500 [00:03<08:10,  1.01it/s]

{"status_code": 200, "request_id": "9f0e527a-1cfa-9508-90a7-fdaea0847b70", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "要找出列表 `[a, b, c, d]` 的最后一个元素，我们需要理解列表的基本概念。在列表中，最后一个元素是序列中的倒数第一个位置的元素。因此，对于列表 `[a, b, c, d]`，其最后一个元素是 `d`。\n\n### 答案:\n答案是：D"}}]}, "usage": {"input_tokens": 138, "output_tokens": 72, "total_tokens": 210}}
{"status_code": 200, "request_id": "a7cd60d5-64be-9e60-bf7b-2d2f0b1f2bf4", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "分析这个问题，我们需要理解列表操作的过程。给定的第一个列表是 [a, b, c]，第二个列表是 [d, e]。题目描述的操作是将第二个列表的元素追加到第一个列表的末尾，形成一个新的列表。\n\n根据这个描述，新列表应该包含原始列表的所有元素，再加上第二个列表的所有元素。因此，新列表应该是 [a, b, c, d, e]。\n\n选项对照：\n- A. [a, b, c, d] 只包含了原始列表和第二个列表的一部分元素。\n- B. [a, d, e, b, c] 元素顺序错误，并且不完整。\n- C. [a, b, c, d, e] 正确地包含了所有元素，按照正确的顺序。\n- D. [d, e, a, b, c] 元素顺序错误，并且不完整。\n\n

...

{"status_code": 200, "request_id": "d0bb5d33-f146-97e3-b0e3-90fbbf46f95f", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "为了回答这个问题，我们需要理解 \"sumn\" 函数的定义。根据描述，“sumn”函数接收一个非负整数作为参数，并返回从1到该整数的所有自然数的和。\n\n对于 \"sumn(3, Y)\" 的调用，我们正在询问从1到3的所有自然数的和，也就是 1 + 2 + 3。这个和等于 6。\n\n因此，答案是：\n\n答案是：B. 6"}}]}, "usage": {"input_tokens": 276, "output_tokens": 98, "total_tokens": 374}}


Processing tasks: 100%|█████████▉| 1327/1328 [09:19<00:01,  1.21s/it]

{"status_code": 200, "request_id": "1d7b121c-2a48-91a1-a234-2c9619ea9d5b", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "为了回答这个问题，我们首先需要理解 `sumn` 函数的定义。根据题目描述，`sumn` 函数接受一个非负整数作为参数，并返回从 1 到该整数的所有自然数的和。\n\n给定参数 `9`，我们需要计算从 1 到 9 的所有自然数的和：\n\n\\[ \\text{Sum} = 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 \\]\n\n\\[ \\text{Sum} = 45 \\]\n\n因此，`sumn(9, Y)` 的结果应该是 `45`。\n\n答案是：C. 45"}}]}, "usage": {"input_tokens": 279, "output_tokens": 147, "total_tokens": 426}}


Processing tasks: 100%|██████████| 1328/1328 [09:19<00:00,  2.37it/s]

{"status_code": 200, "request_id": "abd617c6-14c7-9cee-aae8-6e6d3a75b2da", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "为了正确地解决问题，我们需要考虑如何遵循给定的规则来移动盘子。首先，我们要确保每次移动后盘子的大小顺序不被破坏，并且遵循“不能将一个较大的盘子放在一个较小的盘子上”的规则。\n\n从左柱到右柱的过程中，我们需要确保中间柱（auxiliary）作为临时存储区。因此，正确的策略是在移动一个盘子到右柱之前，先将其移动到中间柱，然后从左柱移除更大的盘子，最后再将先前移到中间柱的盘子移动回右柱。\n\n基于这些原则，我们可以推断出正确的步骤。考虑到最小的盘子在最上面，最大的在最下面，我们需要确保每次移动都保持这种大小顺序。因此，正确的步骤是从左柱开始移动最小的盘子到中间柱，然后将下一个更大的盘子从左柱移除并放到右柱，接着再将最小的盘子从中间柱移动到右柱，以此类推。\n\n根据这个逻辑，正确的步骤应该是这样的：\n\n1. 将顶盘从左柱移动到中柱（这使得中柱上的盘子保持最小的在上，最大的在下）\n2. 将顶盘从左柱移动到右柱（这使得右柱上的盘子保持最小的在上，最大的在下）\n3. 将顶盘从中柱移动到右柱（这使得右柱上的盘子保持最小的在上，最大的在下）\n\n接下来的步骤需要重复这个过程，直到所有盘子都被移动到右柱。所以，正确的步骤是选项 B：\n\n答案是：B"}}]}, "usage": {"input_tokens": 642, "output_tokens": 338, "total_tokens": 980}}
1313
All tasks finished!


In [13]:
def has_complete_answer(questions):
    # 这里假设完整答案的判断逻辑是：每个question都有一个'answer'键
    for question in questions:
        if 'answer' not in question:
            return False
    return True

def filter_problems(data):
    result = []
    problem_set = set()

    for item in data:
        # print('处理的item' ,item)
        problem = item['problem']
        if problem in problem_set:
            # 找到已存在的字典
            for existing_item in result:
                if existing_item['problem'] == problem:
                    # 如果当前字典有完整答案，替换已存在的字典
                    if has_complete_answer(item['questions']):
                        existing_item['questions'] = item['questions']
                        existing_item['id'] = item['id']
                    break
        else:
            # 如果当前字典有完整答案，添加到结果列表
            if has_complete_answer(item['questions']):
                result.append(item)
                problem_set.add(problem)

    return result

return_list = filter_problems(return_list)
# 排序工作 通过id字段后三位代表序号
sorted_data = sorted(return_list, key=lambda x: int(str(x['id'])[-3:]))
print(sorted_data)

[{'problem': '有一群人和一些食物类型。下列是关于这些个体和食物的已知信息：\n\n1. 鸡肉是一种食物。\n2. 苹果是一种食物。\n3. 如果X吃了Y，且X活着，则Y是一种食物。\n4. Bill存活。\n5. Bill吃了花生。\n6. John吃所有食物。\n7. Sue吃所有Bill吃的食物。\n8. John喜欢所有食物。\n\n根据以上信息，回答以下选择题：', 'questions': [{'question': '选择题 1：\n谁喜欢吃花生？', 'options': ['Bill', 'Sue', 'John', 'None of the above'], 'answer': 'D'}], 'id': 'round1_test_data_000'}, {'problem': '设有两个列表操作，第一个列表中包含若干元素，并且可以将第二个列表的元素追加到第一个列表的末尾形成一个新的列表。根据这个操作，请回答以下问题：', 'questions': [{'question': '选择题 1：\n当第一个列表是[a, b, c]，第二个列表是[d, e]时，新的列表是什么？', 'options': ['[a, b, c, d]', '[a, d, e, b, c]', '[a, b, c, d, e]', '[d, e, a, b, c]'], 'answer': 'C'}, {'question': '选择题 2：\n如果新列表是[a, b, c, d, e]，而第二个列表是[d, e]，那么第一个列表是什么？', 'options': ['[a]', '[a, b]', '[a, b, c]', '[b, c]'], 'answer': '\n'}, {'question': '选择题 3：\n当第一个列表是[a, b, c]，新的列表是[a, b, c, d, e]，那么第二个列表是什么？', 'options': ['[d, e]', '[b, c, d]', '[c, d, e]', '[b, d, e]'], 'answer': 'A'}], 'id': 'round1_test_data_001'}, {'problem': '有一个列表，找出该列表的最后一个元素。\n\n下列选项中哪个是列表 `[a, b, c, d]`